# GFD中每个灾害事件的duration和不平等性
Chongyang Du, Jan 9, 2024


首先识别每个灾害事件覆盖的子行政单元
生成每个行政单元在此次灾害事件下的duration(暴露人群人均duration)

分析人均duration和人均收入或消费之间的相关性，期待的结论：duration越长的地区，越贫困的地区

In [1]:
import ee
# ee.Authenticate()


Successfully saved authorization token.


GFD上所有洪涝事件的ID

In [5]:
# Chongyang Du
# Jan 2, 2024

# use this template to run the flood stats functions (area, population, etc.)
# on an image collection and export results as a .tif

import ee
import geemap
ee.Initialize()

# from flood_stats import pop_utils
import time, csv

def get_flood_pop_pixel(flood_img):
    """
    Args:
        floodImage : the standard Earth Engine Image object outputted by the map_DFO_event function
        roiGEO : the region of interest as an Earth Engine Geometry object

    Returns:
    -an image of people in the mapped flood from WorldPop data

    """
    import ee
    import geemap
    ee.Initialize()
    # ee.Authenticate()
    roi_geo = flood_img.geometry()

    # Import the LandScan image collection & permannt water mask
    perm_water = ee.Image("JRC/GSW1_0/GlobalSurfaceWater").select("transition").eq(1).unmask()
    # print('Band names:', flood_img.bandNames().getInfo())
    def maskImages(img):
        non_flood = img.select("flooded")
        water_mask = non_flood.multiply(perm_water.neq(1))
        return img.select("flooded").mask(water_mask)

    def durationImages(img):
            non_flood = img.select("flooded")
            water_mask = non_flood.multiply(perm_water.neq(1))
            return img.select("duration").mask(water_mask)

    # Extract the final flood extent image data as its own variable for analysis
    # flood_extent = maskImages(ee.Image(flood_img.select("flooded")))
    # flood_duration = durationImages(ee.Image(flood_img.select("flood_duration")))
    flood_extent = maskImages(ee.Image(flood_img))
    flood_duration = durationImages(ee.Image(flood_img))


    # # get pop from projects/global-flood-db/landscan
    # # Get event year, match with the population year and clip to study are
    # pop_all = ee.ImageCollection("projects/global-flood-db/landscan")
    # event_year = ee.Date(flood_img.get('began')).get('year')
    # pop_img = ee.Image(pop_all.filterMetadata('year', 'equals', event_year)\
    #                 .first()).clip(roi_geo)
    # pop_img = pop_img.updateMask(pop_img.gte(0)) # mask out bad data with negative values

    # get pop from JRC/GHSL/P2016/POP_GPW_GLOBE_V1
    pop_all = ee.ImageCollection("JRC/GHSL/P2016/POP_GPW_GLOBE_V1")
    # Get event year to match with the population year
    # pop_img = ee.Image(pop_all.filterMetadata('system:index', 'equals', '2000')\
    #                 .first()).clip(roi_geo)
    pop_img = ee.Image(pop_all.filterMetadata('system:index', 'equals', '2015')\
                    .first()).clip(roi_geo)


    # Mask the world population dataset using the flood extent layer
    pop_masked = pop_img.updateMask(flood_extent)

    pop_duration = pop_masked.multiply(flood_duration)
    return flood_extent, flood_duration, pop_img, pop_masked, pop_duration

In [ ]:
# identify the administrative region covered by each flood event
import ee
ee.Initialize()

import time, csv
# from flood_stats import pop_utils

# Image Collection of flood maps, each needs layer called "flooded" that
# is 1 = flooded, 0 = not flooded
gfd = ee.ImageCollection('projects/global-flood-db/gfd_v3').filterMetadata('id','greater_than',1)
# 1500
# Create Error Log file
log_file = "../error_logs/event_stats/pop_error_log_{0}.csv".format(time.strftime("%d_%m_%Y"))
with open(log_file,"w", newline='') as out_file:
    wr = csv.writer(out_file)
    wr.writerow(["error_type", "dfo_id", "error_message"])

# Create list of events from input fusion table
event_ids = ee.List(gfd.aggregate_array('id')).sort()
id_list = event_ids.getInfo()
id_list = [int(i) for i in id_list]
print(len(id_list))
print(id_list)

In [3]:
first_image = gfd.first()

# Retrieve information about the first image as a dictionary.
first_image_info = first_image.getInfo()

# Now you can print or inspect 'first_image_info' to see all properties.
print(first_image_info['properties'])

# If you want to see all property names (keys) of the first image:
property_names = first_image_info['properties'].keys()
print(property_names)

print(first_image_info['properties']['cc'])

{'dfo_centroid_y': -31.268059, 'dfo_main_cause': 'Monsoonal rain', 'gfd_country_name': "['AUSTRALIA']", 'dfo_centroid_x': 143.6978, 'glide_index': 'NA', 'slope_threshold': 5, 'dfo_severity': 2, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[139.6005812730815, -17.518267010104264], [138.4040674236775, -17.51707155596811], [137.20519449393421, -17.518262275331107], [137.20471106885168, -37.68095660034486], [139.99941925183816, -37.680961429261664], [141.9936089883675, -37.680961430234674], [143.98779872401195, -37.680961424329006], [145.58315055189232, -37.68096143983046], [147.97617824298896, -37.680961381848334], [149.97321059715983, -37.68095662726775], [149.97272709473495, -17.518262280872825], [148.77385413322452, -17.51707158835981], [147.57734028086537, -17.518267026312603], [146.38082649061207, -17.517071608501123], [145.18431256933783, -17.518267014811364], [143.98779872401195, -17.51707157652277], [142.39244698359286, -17.51707158759303], [140.79709518683185, -17.5

对每个洪涝灾害，识别覆盖的行政单元

In [6]:
import geopandas as gpd
from shapely.geometry import shape
# Load the shapefile

gdf = ee.FeatureCollection("projects/ee-dcy0910/assets/GSAP2")

# # Convert GeoSeries to list of shapely geometries
# gdf['geometry'] = gdf['geometry'].apply(lambda x: shape(x))
# id_list = [2753]
for event_id in id_list:
    # Get event date range, they can be passed as Strings
    flood_event = ee.Image(gfd.filterMetadata('id', 'equals', event_id).first())
    flood_event_info = flood_event.getInfo()
    # print(flood_event_info['properties']['gfd_country_name'])
    if event_id == 2117:
        flood_event_info['properties']['cc'] = 'PRT, ESP, GIB, FRA, AND, ITA, MCO, UKR, ROU, POL, DEU, CZE, AUT, CHE, LIE, LUX, NLD, BEL, JEY, GGY'
    elif event_id == 3494:
        flood_event_info['properties']['cc'] = 'THA, CHN, LAO, KHM, VNM'
    elif event_id == 4009:
        flood_event_info['properties']['cc'] = 'GBR, IMN'
    elif event_id == 4319:
        flood_event_info['properties']['cc'] = 'GBR, IRL, IMN'
    elif event_id == 4632:
        flood_event_info['properties']['cc'] = 'THA, MMR, BGD, IND, CHN'
    elif event_id == 4640:
        flood_event_info['properties']['cc'] = 'IND, CHN, PAK, NPL'
    elif event_id == 4645:
        flood_event_info['properties']['cc'] = 'IND, CHN, AFG, PAK, NPL'
    elif event_id == 4652:
        flood_event_info['properties']['cc'] = 'THA, MMR, LAO, KHM, VNM'
    elif event_id == 4653:
        flood_event_info['properties']['cc'] = 'CHN, LAO, KHM, VNM'
    elif event_id == 4654:
        flood_event_info['properties']['cc'] = 'CHN, KAZ, RUS, MNG'
    elif event_id == 4662:
        flood_event_info['properties']['cc'] = 'BRA, VEN, GUY, COL'
    elif event_id == 4665:
        flood_event_info['properties']['cc'] = 'MMR, BGD, IND, CHN, BTN, NPL'
    elif event_id == 4666:
        flood_event_info['properties']['cc'] = 'MMR, BGD, IND, CHN'
    elif event_id == 4667:
        flood_event_info['properties']['cc'] = 'BEN, NGA, CMR, TCD, NER'
    elif event_id == 4676:
        flood_event_info['properties']['cc'] = 'USA'
    elif event_id == 4683:
        flood_event_info['properties']['cc'] = 'BFA, TGO, GHA, CIV, BEN'
    elif event_id == 4695:
        flood_event_info['properties']['cc'] = 'USA, MEX'
    elif event_id == 4703:
        flood_event_info['properties']['cc'] = 'BRA, BOL, PRY, ARG'
    elif event_id == 4704:
        flood_event_info['properties']['cc'] = 'THA, LAO, KHM, VNM'
    elif event_id == 4711:
        flood_event_info['properties']['cc'] = 'USA, MEX'

    # 4645. Country: ['IND', 'CHN', 'AFG', 'PAK', 'In dispute CHN/IND', 'NPL']
    # 4652. Country: ['THA', 'MMR', 'LAO', 'KHM', 'VNM']
    # 4653. Country: ['CHN', 'LAO', 'KHM', 'VNM']
    # 4654. Country: ['CHN', 'KAZ', 'RUS', 'MNG']
    # 4662. Country: ['BRA', 'VEN', 'GUY', 'COL']
    # 4665. Country: ['MMR', 'BGD', 'IND', 'CHN', 'BTN', 'In dispute BHUTAN/CHN', 'NPL']
    # 4666. Country: ['MMR', 'BGD', 'IND', 'CHN']
    # 4667. Country: ['BEN', 'NGA', 'CMR', 'TCD', 'NER']
    # 4676. Country: ['USA']
    # 4683. Country: ['BFA', 'TGO', 'GHA', "CIV", 'BEN', 'In dispute BEN/BFA']
    # 4695. Country: ['USA', 'MEX']
    # 4703. Country: ['BRA', 'BOL', 'PRY', 'ARG']
    # 4704. Country: ['THA', 'LAO', 'KHM', 'VNM']
    # 4711. Country: ['USA', 'MEX']




    # Check if 'cc' exists in the properties
    if 'cc' in flood_event_info['properties']:
        event_coun_code = flood_event_info['properties']['cc']
        print("DFO {0} is in {1}".format(event_id, event_coun_code))
        

    else:
        # 'cc' not found, so print 'gfd_country_name' and event_id
        gfd_country_name = flood_event_info['properties'].get('gfd_country_name', 'Unknown Country Name')
        print("Property 'cc' not found for event ID {}. Country: {}".format(event_id, gfd_country_name))
    
    country_codes_list = event_coun_code.split(', ')

    # try:
    # Calculate flood stats
    # flood_stats = pop_utils.getFloodPopbyCountry_GHSLTimeSeries(flood_event)
    flood_extent, flood_duration, pop_img, pop_masked, pop_duration = get_flood_pop_pixel(flood_event)
    # index = flood_stats.get("id").getInfo()

    # band_names = pop_masked.bandNames()
    # # Retrieve the band names as a Python list (makes a call to the Earth Engine API).
    # band_names_list = band_names.getInfo()
    # print("Band names:", band_names_list)

    # Filter the regions covered by the flood event
    # gdf_coun = gdf[gdf['code'].isin(country_codes_list)]
    gdf_coun = gdf.filter(ee.Filter.inList('code', ee.List(country_codes_list)))

    covered_regions = gdf_coun.filterBounds(flood_extent.geometry())
    numberOfRegions = covered_regions.size()

    # Use getInfo() to retrieve the number and print it
    print("found {} regions covered by the flood event".format(numberOfRegions.getInfo()))

    covered_regions_fea_coll = covered_regions
    # Get area of flood in the scale of the flood map
    flood_area_img = flood_extent.multiply(ee.Image.pixelArea())
    map_scale = flood_extent.projection().nominalScale()
    
    flood_extent, flood_duration, pop_img, pop_masked, pop_duration
    pop_scale = pop_img.projection().nominalScale()

    pop_masked_scale = pop_masked.projection().nominalScale()

    pop_duration_scale = pop_duration.projection().nominalScale()

    def get_regions_dura(ft):

        pop_sum = pop_img.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = ft.geometry(),
        scale = pop_scale,
        maxPixels = 1e9)

        area_sum= flood_area_img.reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= ft.geometry(),
        scale= map_scale,
        maxPixels= 1e9)

        pop_exposed= pop_masked.reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= ft.geometry(),
        scale= pop_masked_scale,
        maxPixels= 1e9)

        pop_duration_mask = pop_duration.reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= ft.geometry(),
        scale= pop_duration_scale,
        maxPixels= 1e9)

        pop_init_sum = pop_sum.get("population_count")
        pop_exposed_sum = pop_exposed.get("population_count")
        pop_duration_mask_sum = pop_duration_mask.get("population_count")
        area = area_sum.get("flooded")
        sys_id = ee.String(flood_event.get('system:index')).getInfo()

        # Calculate the ratio of pop_duration_mask_sum to pop_exposed_sum
        # Ensure that pop_exposed_sum is not zero to avoid division by zero error
        pop_duration_mean = ee.Algorithms.If(
            ee.Number(pop_exposed_sum).gt(0),
            ee.Number(pop_duration_mask_sum).divide(pop_exposed_sum),
            None
        )
        return ee.Feature(None, {
            "system:index": sys_id,
            "id": event_id,
            "Country_code": ft.get("code"),
            "geo_code": ft.get("get_code2"),
            "level": ft.get("level"),
            "GSAP2_mean": ft.get("GSAP2_mean"),
            "GSAP2_medi": ft.get("GSAP2_medi"),
            "GSAP2_po_1": ft.get("GSAP2_po_1"),
            "GSAP2_po_2": ft.get("GSAP2_po_2"),
            "GSAP2_poor": ft.get("GSAP2_poor"),
            "GSAP2_gini": ft.get("GSAP2_gini"),
            "GSAP2_thei": ft.get("GSAP2_thei"),
            "pop_all": pop_init_sum,
            "pop_exposed": pop_exposed_sum,
            "pop_duration_sum": pop_duration_mask_sum,
            "pop_duration_mean": pop_duration_mean,
            "Area": area
        })
    try:
        regions_stats = ee.FeatureCollection(covered_regions_fea_coll).map(get_regions_dura)
        regions_stats = ee.FeatureCollection(regions_stats).set({"id":event_id})
        print("calculated results, exporting results for DFO {0}...".format(int(event_id)))

    except Exception as e:
        s = str(e)
        with open(log_file,"w", newline='') as out_file:
            wr = csv.writer(out_file)
            wr.writerow(["Calculation Error", event_id, s])
        print("Calculation Error {0} - Cataloguing and moving onto next event".format(event_id))
        print("-------------------------------------------------")

    # Export results
    try:
        task = ee.batch.Export.table.toDrive(
            collection = regions_stats,
            description = 'duration_state_regions_{0}'.format(str(int(event_id))),
            folder='Data/GFD/region_duration/',
            fileNamePrefix = 'duration_state_regions_{0}'.format(str(int(event_id))),
            fileFormat = 'CSV')

        task.start()

    except Exception as e:
        s = str(e)
        with open(log_file,"ab") as out_file:
            wr = csv.writer(out_file)
            wr.writerow(["Export Error", event_id, s])
        print("Export Error DFO {0} - Cataloguing and moving onto next event".format(event_id))
        print("-------------------------------------------------")

print('Done!')

    



DFO 2753 is in USA, MEX, CUB, DOM, GTM, NIC, HND, HTI, JAM, BLZ
found 64 regions covered by the flood event
calculated results, exporting results for DFO 2753...
Done!


In [ ]:
# Chongyang Du
# Jan 2, 2024

# use this template to run the flood stats functions (area, population, etc.)
# on an image collection and export results as a .tif

import ee
import geemap
ee.Initialize()

# from flood_stats import pop_utils
import time, csv

def get_flood_pop_pixel(flood_img):
    """
    Args:
        floodImage : the standard Earth Engine Image object outputted by the map_DFO_event function
        roiGEO : the region of interest as an Earth Engine Geometry object

    Returns:
    -an image of people in the mapped flood from WorldPop data

    """
    import ee
    import geemap
    ee.Initialize()
    # ee.Authenticate()
    roi_geo = flood_img.geometry()

    # Import the LandScan image collection & permannt water mask
    perm_water = ee.Image("JRC/GSW1_0/GlobalSurfaceWater").select("transition").eq(1).unmask()
    # print('Band names:', flood_img.bandNames().getInfo())
    def maskImages(img):
        non_flood = img.select("flooded")
        water_mask = non_flood.multiply(perm_water.neq(1))
        return img.select("flooded").mask(water_mask)

    def durationImages(img):
            non_flood = img.select("flooded")
            water_mask = non_flood.multiply(perm_water.neq(1))
            return img.select("duration").mask(water_mask)

    # Extract the final flood extent image data as its own variable for analysis
    # flood_extent = maskImages(ee.Image(flood_img.select("flooded")))
    # flood_duration = durationImages(ee.Image(flood_img.select("flood_duration")))
    flood_extent = maskImages(ee.Image(flood_img))
    flood_duration = durationImages(ee.Image(flood_img))


    # # get pop from projects/global-flood-db/landscan
    # # Get event year, match with the population year and clip to study are
    # pop_all = ee.ImageCollection("projects/global-flood-db/landscan")
    # event_year = ee.Date(flood_img.get('began')).get('year')
    # pop_img = ee.Image(pop_all.filterMetadata('year', 'equals', event_year)\
    #                 .first()).clip(roi_geo)
    # pop_img = pop_img.updateMask(pop_img.gte(0)) # mask out bad data with negative values

    # get pop from JRC/GHSL/P2016/POP_GPW_GLOBE_V1
    pop_all = ee.ImageCollection("JRC/GHSL/P2016/POP_GPW_GLOBE_V1")
    # Get event year to match with the population year
    # pop_img = ee.Image(pop_all.filterMetadata('system:index', 'equals', '2000')\
    #                 .first()).clip(roi_geo)
    pop_img = ee.Image(pop_all.filterMetadata('system:index', 'equals', '2015')\
                    .first()).clip(roi_geo)


    # Mask the world population dataset using the flood extent layer
    pop_masked = pop_img.updateMask(flood_extent)

    pop_duration = pop_masked.multiply(flood_duration)
    return flood_extent, flood_duration, pop_img, pop_masked, pop_duration



# Image Collection of flood maps, each needs layer called "flooded" that
# is 1 = flooded, 0 = not flooded
gfd = ee.ImageCollection('projects/global-flood-db/gfd_v3')
# gfd = ee.ImageCollection('projects/global-flood-db/gfd_v3').filterMetadata('id','greater_than',4335)

# Create Error Log file
log_file = "error_logs/event_stats/pop_error_log_{0}.csv".format(time.strftime("%d_%m_%Y"))
with open(log_file,"w", newline='') as out_file:
    wr = csv.writer(out_file)
    wr.writerow(["error_type", "dfo_id", "error_message"])

# Create list of events from input fusion table
event_ids = ee.List(gfd.aggregate_array('id')).sort()
id_list = event_ids.getInfo()
id_list = [int(i) for i in id_list]

# for event_id in id_list:
# event_id = id_list[0]
event_id = 4444
# Get event date range, they can be passed as Strings
flood_event = ee.Image(gfd.filterMetadata('id', 'equals', event_id).first())

# try:
# Calculate flood stats
# flood_stats = pop_utils.getFloodPopbyCountry_GHSLTimeSeries(flood_event)
flood_extent, flood_duration, pop_img, pop_masked, pop_duration = get_flood_pop_pixel(flood_event)
# index = flood_stats.get("id").getInfo()
print("calculated results, exporting results for DFO {0}...".format(event_id))

# 定义感兴趣的区域
region = pop_img.geometry()

# 设置下载参数
output_dir = '/download/'
scale = pop_img.projection().nominalScale()  # 设置所需的空间分辨率
file_format = 'GeoTIFF'  # 设置所需的文件格式

# 下载图像集合

filename = "Data/GFD/duration_pop_extent/flooded_{0}.tif".format(event_id)
geemap.download_ee_image(flood_extent, scale=scale, region=region, filename=filename, crs="EPSG:4326")

filename = "Data/GFD/duration_pop_extent/flood_duration_{0}.tif".format(event_id)
geemap.download_ee_image(flood_duration, scale=scale, region=region, filename=filename, crs="EPSG:4326")

filename = "Data/GFD/duration_pop_extent/pop_{0}.tif".format(event_id)
geemap.download_ee_image(pop_img, scale=scale, region=region, filename=filename, crs="EPSG:4326")

filename = "Data/GFD/duration_pop_extent/pop_flooded_{0}.tif".format(event_id)
geemap.download_ee_image(pop_masked, scale=scale, region=region, filename=filename, crs="EPSG:4326")

filename = "Data/GFD/duration_pop_extent/pop_duration_{0}.tif".format(event_id)
geemap.download_ee_image(pop_duration, scale=scale, region=region, filename=filename, crs="EPSG:4326")


print('Done!')


